In [7]:
# From Brad Garton's gen patch
# Max/help/examples/gen/gen~.flute

In [8]:
import numpy as np
import time
import soundfile as sf

In [9]:
tube_length= 36
embochure = 20
fs = 44100
nbr_de_sec = 3
duration = fs*nbr_de_sec

In [10]:
def noise(duration):
    y = (np.random.rand(duration,1)-0.5)*2
    return y

def normalize(array):
    min = np.min(array)
    max = np.max(array)
    normalized_array = 2*(array-min)/(max-min) - 1
    return normalized_array

In [11]:
# Cook-Garton 1992

# Noise ノイズ
burst_noise = noise(duration)

# Variables 変数
output = np.empty(duration)  # Preallocate output array for efficiency
delayTube1 = np.empty(duration)
delayTube2 = np.empty(duration)
delayEmb = np.empty(duration)

feedback1 = 0
feedback2 = 0

delayIR = 0
start = 0
end = 0
noiseamp = 0.3

start = time.time()
for i in range(duration):
    instantSample = burst_noise[i] * noiseamp
    instantSample += 1.

    # + Feedback
    if i !=0:
        instantSample += feedback1
    
    # Delay 100
    delayEmb[i] = instantSample
    if i > embochure:
        instantSample = delayEmb[i-embochure]

    # Nonlinearity
    instantSample = (instantSample**3) - instantSample
    instantSample *= 0.4

    # + Feedback
    if i !=0:
        instantSample += feedback2
    instantSample_to_output = instantSample

    # IR Filter 無限インパルス応答フィルター
    instantSample = instantSample * 0.7 + delayIR * 0.3
    delayIR = instantSample

    # Delay 100
    delayTube1[i] = instantSample
    if i > tube_length:
        instantSample = delayTube1[i-tube_length]

    # Feedbacks フィードバック
    feedback1 = instantSample * -0.35
    feedback2 = instantSample * 0.9

    # Output
    output[i] = instantSample_to_output * 0.5

normalized_output = normalize(output)
end = time.time()

In [12]:
# write output to file for listening
sf.write('output_Cook-Garton92.wav', normalized_output, fs)

elapsed_time = end - start
print(elapsed_time)

0.5795340538024902
